In [2]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time
import json
import pandas as pd

In [ ]:
#function to create an API request based on URL and  parameters
def get_request(url, parameters=None):
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
        
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(1)



#using the function to get all data id's which include appid and app name 
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
parameters = {"request": all}
data_ids = get_request(url, parameters=parameters)
print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")

data_ids

In [7]:
#Code to collect all app id's in one library
app_ids = []
print("Creating a list of all IDs currently available on Steam")
for item in data_ids['applist']['apps']:
    app_ids.append(item['appid'])

print("List created sucessfully, the total number of IDs available on Steam is:", len(app_ids),".")
print("Call upon 'app_ids' to preview the full list of extracted IDs.")



Creating a list of all IDs currently available on Steam
List created sucessfully, the total number of IDs available on Steam is: 138039 .
Call upon 'app_ids' to preview the full list of extracted IDs.


In [ ]:
# example code that collects the details of 1 app by appid  
url = "http://store.steampowered.com/api/appdetails/"
parameters = {"appids": "10"}
app_data = get_request(url, parameters=parameters)
app_data


In [ ]:
#it finaly loops
#however it gives some succes and some false values. so we need to build in a check if succes = false -> then next. if succes = True -> write the data to a Csv 
for item in app_ids:
    url_appdetails= "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {item}}
    app_details = get_request(url_appdetails, parameters=parameters)
    
    print(app_details)

    app_details
    


In [ ]:
## collecting the data with a for loop that checks if the response was a succes = True. and than takes the data attribute 
details = {} 

counter = 1   
for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)
    id = str(i)
    print("Processing data pertaining to Steam ID:"+id)
    response = app_data[id]
    succes = response["success"]
    if (succes): 
        data_json = response["data"]
        try:
            app_id = data_json["steam_appid"]
            name = data_json["name"]
            prod_type = data_json["type"]
            freemium = data_json["is_free"]
            price = data_json["price_overview"]["final_formatted"]
            discount = data_json["price_overview"]["discount_percent"]
            developer = data_json["developers"]
            publisher = data_json["publishers"]
            desc = data_json["detailed_description"]
            short_desc = data_json["short_description"]
            try:
                release_date = data_json["release_date"]["date"]
            except:
                release_date = "TBA"
            try:
                win = data_json["platforms"]["windows"]
            except:
                win = ""
            try:
                mac = data_json["platforms"]["mac"]
            except:
                mac = ""
            try:
                lin = data_json["platforms"]["linux"]
            except:
                lin = ""
            try:
                achievements = data_json["achievements"]["total"]
            except:
                achievements = "0"
            try:
                reviews = data_json["recommendations"]
            except:
                reviews = ""
            try:
                metacritic = data_json["metacritic"]
            except:
                metacritic = "not available"
            try:
                base_app = data_json["fullgame"]["appid"]
            except:
                base_app = "none"
            scrape_datetime = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            details[app_id]=[name],[release_date],[freemium],[prod_type],[price],[discount],[developer],[publisher],[desc],[short_desc],[achievements],[win],[mac],[lin],[reviews],[metacritic],[base_app],[scrape_datetime]
        except:
            pass
        counter = counter + 1 
    if (counter > 10000):
        break           


In [ ]:
details

In [ ]:
### example to create a dictionary that checks if the id is already in the dictionary and then if not in dictionary get data 

"""
details = {}

for i in app_ids:
  id = str(i)
  if id not in details:
     // get data

     if(success):
       // put data in details
       details[id] = data

"""     

In [ ]:
#not relevant yet 
#trying to make a function out of it


"""
this is a comment 

def Get_tha_data(attributes, limit):
 
counter = 1   
for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)       
    id = str(i)
    response = app_data[id]
    succes = response["success"]
    if (succes): 
        data_json = response["data"]
        print(data_json)
        counter = counter + 1 
    if (counter > limit):
        break           
"""